In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the CSV files
train_data_df = pd.read_csv('Released_Data/train_data.csv')
super_classes_df = pd.read_csv('Released_Data/super_classes_mapping.csv')
sub_classes_df = pd.read_csv('Released_Data/sub_classes_mapping.csv')

# Rename
super_classes_df.rename(columns={'class': 'superclass_name'}, inplace=True)
sub_classes_df.rename(columns={'class': 'subclass_name'}, inplace=True)

# Merge the class names with the training data
train_data_df = train_data_df.merge(super_classes_df, left_on='superclass_index', right_on='index', how='left')
train_data_df = train_data_df.merge(sub_classes_df, left_on='subclass_index', right_on='index', how='left')

# Choose either superclass_name or subclass_name based on classification task
train_data_df['class'] = train_data_df['superclass_name']

# Split the dataset into training and validation sets
train_df, validation_df = train_test_split(train_data_df, test_size=0.2)

# Initialize the ImageDataGenerator with EfficientNet's preprocess_input
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Image folder path
image_folder_path = 'Released_Data/train_shuffle'

# Load and preprocess images for training
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_folder_path,
    x_col='image',
    y_col='class',
    class_mode='categorical',
    target_size=(300, 300),
    batch_size=32
)

# Load and preprocess images for validation
validation_generator = datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory=image_folder_path,
    x_col='image',
    y_col='class',
    class_mode='categorical',
    target_size=(300, 300),
    batch_size=32
)

# Load pre-trained EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model
base_model.trainable = False

# Add custom top layers
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(len(super_classes_df), activation='softmax')(x) # Number of classes
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Found 5177 validated image filenames belonging to 3 classes.
Found 1295 validated image filenames belonging to 3 classes.


In [13]:
# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(train_generator, validation_data=validation_generator, epochs=10, batch_size=32)


Epoch 1/10
162/162 [==============================] - 810s 5s/step - loss: 0.8752 - accuracy: 0.5940 - val_loss: 0.7654 - val_accuracy: 0.6749
Epoch 2/10
162/162 [==============================] - 786s 5s/step - loss: 0.7729 - accuracy: 0.6612 - val_loss: 0.7502 - val_accuracy: 0.6819
Epoch 3/10
162/162 [==============================] - 791s 5s/step - loss: 0.7368 - accuracy: 0.6813 - val_loss: 0.7336 - val_accuracy: 0.6981
Epoch 4/10
162/162 [==============================] - 791s 5s/step - loss: 0.7198 - accuracy: 0.6977 - val_loss: 0.7182 - val_accuracy: 0.7073
Epoch 5/10
162/162 [==============================] - 788s 5s/step - loss: 0.7003 - accuracy: 0.7025 - val_loss: 0.7092 - val_accuracy: 0.7066
Epoch 6/10
162/162 [==============================] - 793s 5s/step - loss: 0.6968 - accuracy: 0.7052 - val_loss: 0.6828 - val_accuracy: 0.7120
Epoch 7/10
162/162 [==============================] - 789s 5s/step - loss: 0.6865 - accuracy: 0.7135 - val_loss: 0.6611 - val_accuracy: 0.7336

Epoch 1/10
162/162 [==============================] - 869s 5s/step - loss: 0.7469 - accuracy: 0.6954 - val_loss: 0.6419 - val_accuracy: 0.7514
Epoch 2/10
162/162 [==============================] - 870s 5s/step - loss: 0.5845 - accuracy: 0.7562 - val_loss: 0.6396 - val_accuracy: 0.7537
Epoch 3/10
162/162 [==============================] - 869s 5s/step - loss: 0.5452 - accuracy: 0.7786 - val_loss: 0.7296 - val_accuracy: 0.7205
Epoch 4/10
162/162 [==============================] - 869s 5s/step - loss: 0.5094 - accuracy: 0.8041 - val_loss: 0.5626 - val_accuracy: 0.7853
Epoch 5/10
162/162 [==============================] - 870s 5s/step - loss: 0.4904 - accuracy: 0.8007 - val_loss: 0.5033 - val_accuracy: 0.8085
Epoch 6/10
162/162 [==============================] - 860s 5s/step - loss: 0.4701 - accuracy: 0.8063 - val_loss: 0.4815 - val_accuracy: 0.8154
Epoch 7/10
162/162 [==============================] - 856s 5s/step - loss: 0.4466 - accuracy: 0.8211 - val_loss: 0.5050 - val_accuracy: 0.8147

In [14]:
import os
# Check the directory path
print("Directory exists:", os.path.isdir(image_folder_path))

# Check the first few rows of train_df and validation_df
print("Train DataFrame:\n", train_df.head())
print("Validation DataFrame:\n", validation_df.head())

# Check the length of the DataFrames
print("Length of train_df:", len(train_df))
print("Length of validation_df:", len(validation_df))

# Check a few image file paths
sample_images = train_df['image'].head().tolist()
for img in sample_images:
    file_path = os.path.join(image_folder_path, img)
    print(f"{img} exists:", os.path.isfile(file_path))


Directory exists: True
Train DataFrame:
          image  superclass_index  subclass_index  index_x superclass_name  \
4468  4468.jpg                 2              44        2         reptile   
3498  3498.jpg                 0              28        0            bird   
6148  6148.jpg                 0              30        0            bird   
354    354.jpg                 1              31        1             dog   
4655  4655.jpg                 2              69        2         reptile   

      index_y                                      subclass_name    class  
4468       44              hognose snake, puff adder, sand viper  reptile  
3498       28                                       black grouse     bird  
6148       30                                            vulture     bird  
354        31                                           Shih-Tzu      dog  
4655       69  leatherback turtle, leatherback, leathery turt...  reptile  
Validation DataFrame:
          image  s